In [1]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cmocean
import spatialpandas.dask 
import dask as da
import dask.dataframe as dd
import matplotlib.pyplot as plt
import pandas as pd

import uxarray as ux

import holoviews as hv
from holoviews import opts
from holoviews.operation.datashader import rasterize
import datashader as ds
import datashader.transfer_functions as tf
import geoviews.feature as gf

import spatialpandas as sp

import multiprocessing

from multiprocessing import Pool


In [74]:
#pip install pgpd

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
%load_ext autoreload
%autoreload 2

sys.path.insert(1, '/glade/u/home/philipc/geocat-internal-testing/polyplot/')

from polyplot import poly_plot as pp

# Data

In [3]:
ds_base_path = "/glade/p/cisl/vast/vapor/data/Source/UGRID/NOAA-geoflow/large/"
ds_grid = ux.open_dataset(ds_base_path + "grid.nc")
ds_v1 = xr.open_dataset(ds_base_path + "v1.000000.nc")

Loading initial grid from file:  /glade/p/cisl/vast/vapor/data/Source/UGRID/NOAA-geoflow/large/grid.nc


In [4]:
ds_grid.ds

<xarray.Dataset>
Dimensions:          (nMeshFaces: 3932160, nFaceNodes: 4, nMeshNodes: 6144000, meshLayers: 20)
Coordinates:
    mesh_node_x      (nMeshNodes) float64 ...
    mesh_node_y      (nMeshNodes) float64 ...
Dimensions without coordinates: nMeshFaces, nFaceNodes, nMeshNodes, meshLayers
Data variables:
    mesh             int32 ...
    mesh_face_nodes  (nMeshFaces, nFaceNodes) uint32 ...
    mesh_depth       (meshLayers, nMeshNodes) float64 ...

In [5]:
x = ds_grid.ds['mesh_node_x'].values
y = ds_grid.ds['mesh_node_y'].values
face_nodes = ds_grid.ds['mesh_face_nodes'].values.astype(int)

In [6]:
var_dict = ds_grid.ds_var_names
ugrid_dict = dict((k, v) for k,v in var_dict.items())

In [7]:
# Specify Time and Elevation Slice
t, level = 0, 0

# Data Variable to Plot
v1 = ds_v1['v1'][t][level].values

# PolygonArray Bottleneck

In [15]:
def test_to_list(n):
    poly_array = np.ones((n, 1, 8))
    a = poly_array.tolist()

In [16]:
def test_polygon_array(n):
    poly_array = np.ones((n, 1, 8))
    polygons = sp.geometry.PolygonArray(poly_array.tolist())

In [29]:
n_list = [10**n for n in range(3, 8)]
n_list

[1000, 10000, 100000, 1000000, 10000000]

In [30]:
print("Test: tolist()")
for n in n_list:
    print("\n")
    print("Number of Polygons: {}".format(n))
    %timeit -r 3 -n 5 test_to_list(n) 

Test: tolist()


Number of Polygons: 1000
264 µs ± 72.4 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 10000
2.18 ms ± 101 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 100000
37.4 ms ± 457 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 1000000
446 ms ± 515 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 10000000
4.55 s ± 30.6 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [31]:
print("Test: PolygonArray")
for n in n_list:
    print("\n")
    print("Number of Polygons: {}".format(n))
    %timeit -r 3 -n 5 test_polygon_array(n) 

Test: PolygonArray


Number of Polygons: 1000
1.81 ms ± 115 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 10000
15.9 ms ± 204 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 100000
181 ms ± 2.06 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 1000000
1.95 s ± 8.81 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 10000000
19.7 s ± 86.9 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


# PyGeos

In [49]:
import pygeos as pg
import pyarrow as pa

In [50]:
index = face_nodes.astype(int)
x_coords = x[index]
y_coords = y[index]
n = len(y_coords)
print(x_coords.shape, y_coords.shape, n)

(3932160, 4) (3932160, 4) 3932160


In [51]:
# want [n x 4 x 2] dimensions
polygons = np.zeros((n, 4, 2))
polygons[:, :, 0] = x_coords
polygons[:, :, 1] = y_coords

In [71]:
%%time
geo = pg.polygons(polygons)


CPU times: user 2.45 s, sys: 771 ms, total: 3.22 s
Wall time: 3.23 s


In [70]:
geo

<pygeos.Geometry MULTIPOLYGON (((0 89.192, 4.991 89.238, 5.128 89.301, 0 89....>

In [73]:
out = spatialpandas.geometry.MultiPolygonArray(geo)
df = sp.GeoDataFrame({'geometry': out})

ArrowInvalid: Could not convert <pygeos.Geometry POLYGON ((0 89.192, 4.991 89.238, 5.128 89.301, 0 89.262, 0...> with type pygeos.lib.Geometry: did not recognize Python value type when inferring an Arrow data type

In [48]:
%%time
arr_flat, part_indices = pg.get_parts(geo, return_index=True)
offsets1 = np.insert(np.bincount(part_indices).cumsum(), 0, 0)
arr_flat2, ring_indices = pg.geometry.get_rings(arr_flat, return_index=True)
offsets2 = np.insert(np.bincount(ring_indices).cumsum(), 0, 0)
coords, indices = pg.get_coordinates(arr_flat2, return_index=True)
offsets3 = np.insert(np.bincount(indices).cumsum(), 0, 0)

coords_flat = coords.ravel()
offsets3 *= 2

# create a pyarrow array from this
_parr3 = pa.ListArray.from_arrays(pa.array(offsets3), pa.array(coords_flat))
_parr2 = pa.ListArray.from_arrays(pa.array(offsets2), _parr3)
parr = pa.ListArray.from_arrays(pa.array(offsets1), _parr2)

out = spatialpandas.geometry.MultiPolygonArray(parr)

CPU times: user 4.23 s, sys: 144 ms, total: 4.38 s
Wall time: 4.38 s


In [46]:
df = sp.GeoDataFrame({'geometry': out})

In [47]:
df

,geometry
0,"MultiPolygon([[[0.0, 89.19234422049671, 4.9905..."
1,"MultiPolygon([[[4.990573301226268, 89.23763239..."
2,"MultiPolygon([[[16.03568682344074, 89.30480405..."
3,"MultiPolygon([[[28.84955151877949, 89.34145656..."
4,"MultiPolygon([[[0.0, 89.26207414750719, 5.1276..."
...,...
3932155,"MultiPolygon([[[36.59656383981396, -26.9269271..."
3932156,"MultiPolygon([[[36.40526181302508, -26.7285724..."
3932157,"MultiPolygon([[[36.471060549948376, -26.751452..."
3932158,"MultiPolygon([[[36.595865107639355, -26.794741..."


In [37]:
out = sp.geometry.MultiPolygonArray(pa.ListArray(arr_flat))

ArrowInvalid: Could not convert <pygeos.Geometry POLYGON ((0 89.192, 4.991 89.238, 5.128 89.301, 0 89.262, 0...> with type pygeos.lib.Geometry: did not recognize Python value type when inferring an Arrow data type

# PyGeos and PGPD

In [115]:
import pandas as pd
import pygeos as pg
import pgpd

In [102]:
index = face_nodes.astype(int)
x_coords = x[index]
y_coords = y[index]
data = np.mean(data[index], axis=1)
n = len(y_coords)

print(n, len(data))

3932160 3932160


In [78]:
# want [n x 4 x 2] dimensions
polygons = np.zeros((n, 4, 2))
polygons[:, :, 0] = x_coords
polygons[:, :, 1] = y_coords

In [106]:
geo = pg.polygons(polygons)

In [119]:
df = pd.DataFrame({"poly": geo, "faces" : data})
df = df.astype({'poly':'geos'})


In [121]:
type(df.geos.to_geopandas(geometry='poly'))

geopandas.geodataframe.GeoDataFrame